In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from functions import import_data
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor


In [3]:
df = import_data("./data/02/cleaned_house_prices.csv", "Id")

Memory usage of dataframe is 0.075 MB
Memory usage after optimization is: 0.04 MB
Decreased by 49.7%


In [ ]:
y = df['target']
X = df.drop('target', axis=1)

In [ ]:
from pycaret import


In [4]:
df.columns

Index(['AgeBins', 'BldgType', 'GrLivArea', 'LotFrontage', 'LotArea',
       'HasFence', 'Has2ndFloor', 'HasBasement', 'HasGarage', 'target'],
      dtype='object')

In [ ]:
cat_cols = []
num_cols = ['Base Liv Area', 'Garage Area', 'Gr Liv Area']
y = df["SalePrice"]
X = df.drop("SalePrice", axis=1)
X_num = X[num_cols]

In [ ]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(X[cat_cols])
X_trans = ohe.transform(X[cat_cols])
X_trans = pd.DataFrame(X_trans, columns=ohe.get_feature_names_out())
X_trans.head()
X = pd.concat([X_trans, X_num], axis=1)
X.head()

In [ ]:
import numpy as np
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
print(train.shape, val.shape, test.shape)

In [ ]:
x_cols = cat_cols + num_cols
y_col = ['SalePrice']

train_x = train[x_cols]
train_y = np.ravel(train[y_col])
val_x = val[x_cols]
val_y = np.ravel(val[y_col])
test_x = test[x_cols]
test_y = np.ravel(test[y_col])

In [ ]:
categorical_pipeline = Pipeline(steps=[
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

full_processor = ColumnTransformer(transformers=[
    ('category', categorical_pipeline, cat_cols)
])

xgbrfr = XGBRegressor()

pipe = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', xgbrfr)
])

In [ ]:
import time
from sklearn.experimental import enable_halving_search_cv #noqa
from sklearn.model_selection import HalvingRandomSearchCV
from xgboost import XGBRFRegressor


xgb.set_config(verbosity=0)
params = {
    'model__n_estimators': [100, 200, 300],
    'model__min_child_weight': [1, 3, 5, 7, 9],
    'model__gamma': [.1*i for i in range(10)],
    'model__subsample': [i/10 for i in range(7, 10)],
    'model__colsample_bytree': [i/10 for i in range(7, 10)],
    'model__max_depth': [3, 5, 7, 9],
    'model__reg_alpha': [1e-5*10**i for i in range(6)],
    'model__reg_lambda': [1e-5*10**i for i in range(6)],
    'model__booster': ['gbtree', "dart", "gblinear"],
    'model__learning_rate': [1e-1*i for i in range(3)],
}

hg = HalvingRandomSearchCV(
    estimator=pipe,
    resource='n_samples',
    param_distributions=params, 
    cv=10, 
    min_resources=50,
    verbose=10, factor=1.01,
    random_state=888,
    scoring="neg_mean_absolute_error",
    return_train_score=True,
    aggressive_elimination=True
)

start = time.time()
hg.fit(train_x, train_y)
print("HalvingRandomSearchCV took %.2f seconds"
      " parameter settings." % ((time.time() - start)))

NameError: name 'import_data' is not defined

In [ ]:
import pyca

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1313 entries, 1 to 1460
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   AgeBins      1313 non-null   int8    
 1   BldgType     1313 non-null   category
 2   GrLivArea    1313 non-null   int16   
 3   LotFrontage  1313 non-null   int16   
 4   LotArea      1313 non-null   int32   
 5   HasFence     1313 non-null   float16 
 6   Has2ndFloor  1313 non-null   float16 
 7   HasBasement  1313 non-null   float16 
 8   HasGarage    1313 non-null   float16 
 9   target       1313 non-null   int32   
dtypes: category(1), float16(4), int16(2), int32(2), int8(1)
memory usage: 38.7 KB


In [1]:
df.columns

NameError: name 'df' is not defined

In [28]:
num_cols = ["GrLivArea", "LotFrontage", "LotArea"]

In [29]:
ohe = OneHotEncoder()
X_out= ohe.fit_transform(X_train[["AgeBins", "BldgType"]])
X_test[["AgeBins", "BldgType"]] = ohe.fit(X_test[["AgeBins", "BldgType"]])

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.fit(X_test[num_cols])


ValueError: Columns must be same length as key

In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_test, label=y_test)

In [ ]:
callbacks = [
    lgb.reset_parameter(lambda x: x * 0.99),
    lgb.early_stopping(10)
]

In [ ]:
model = lgb.LGBMRegressor(
    learning_rate=1e-4,
    n_estimators=1000,
    callbacks=callbacks,
    objective="regression",
    metrics=['rmse']
)

model.fit(dtrain)